In [16]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import pickle

import pandas as pd

import numpy as np

import evaluate

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from torch.utils.data import DataLoader

import torch

import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

In [2]:
# Abrindo o objeto dataframe
with open(r'df-tokenizado.pickle', 'rb') as pickledfile:
    df = pickle.load(pickledfile)

In [3]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')

In [4]:
df['Tokenizadas 2'] = [tokenizer.encode(t, truncation='only_first', padding='max_length', max_length=512) for t in df['Texto tratado']]

In [5]:
y = LabelEncoder().fit_transform(X = df['Assunto'].values.reshape(-1,1))

c:\Users\Lucas Alponti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
metric = evaluate.load("accuracy")

In [7]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

In [8]:
folds = StratifiedKFold(n_splits=10)

In [9]:
splits = folds.split(df['Tokenizadas 2'], y)

In [10]:
list_splits = []

for f, (t_, v_) in enumerate(splits):
    list_splits.append([f, t_, v_])

In [11]:
bertmodel = AutoModelForSequenceClassification.from_pretrained('neuralmind/bert-large-portuguese-cased', num_labels = 5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-large-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
training_args = TrainingArguments(output_dir="teste", eval_strategy="epoch", use_cpu=False, no_cuda=False)

t_ = list_splits[0][1]
v_ = list_splits[0][2]

df_teste = pd.DataFrame.from_dict(
    {'input_ids': df.loc[t_, 'Tokenizadas 2'], 'label': y[t_], 'attention_mask':[[0 if elem == 0 else 1 for elem in row] for row in df.loc[t_, 'Tokenizadas 2']] }
)
hg_dataset_teste = Dataset(pa.Table.from_pandas(df_teste))

df_val = pd.DataFrame.from_dict(
    {'input_ids': df.loc[v_, 'Tokenizadas 2'], 'label': y[v_], 'attention_mask':[[0 if elem == 0 else 1 for elem in row] for row in df.loc[v_, 'Tokenizadas 2']]}
)
hg_dataset_val = Dataset(pa.Table.from_pandas(df_val))

trainer = Trainer(
    model=bertmodel,
    args=training_args,
    train_dataset=hg_dataset_teste,
    eval_dataset=hg_dataset_val,
    compute_metrics=compute_metrics
)

In [22]:
trainer.train()

In [17]:
torch.cuda.is_available()

False